<a id="top_of_page"></a>
# NORNIR BASICS

Working through the initiliasation from Nornir documentation<br>
https://nornir.readthedocs.io/en/latest/tutorial/index.html<br>
(the error in **ln [14]** (on a freshly started kernel) is meant to be there) 


## Table of contents:<br>

[1 Initialiting Nornir](#initializing_nornir)<br>
[2 Inventory](#inventory)<br>
[3 Inheritance](#inheritance)<br>
[4 Filtering](#filtering)<br>
[5 Tasks](#tasks)<br>
[6 Processing Results](#processing_results)<br>
[7 Failed Tasks](#failed_tasks)<br>
[8 Processors](#processors)<br>

<a id="initializing_nornir"></a>
## 1 Initializing Nornir

In [1]:
from nornir import InitNornir

### 1.1 Simple Nornir - Object creation with all files necessary in place

In [2]:
nr = InitNornir(config_file="config.yaml")
# nr

### 1.2 Initialize nornir programmatically without a configuration file
Two dictionaries must be defined: *runner* and *inventory*

In [3]:
nr = InitNornir(
    runner={
        "plugin": "threaded",
        "options": {
            "num_workers": 100,
        },
    },
    inventory={
        "plugin": "SimpleInventory",
        "options": {
            "host_file": "inventory/hosts.yaml",
            "group_file": "inventory/groups.yaml"
        },
    },
)
# nr

### 1.3 Combination of both methods, e.g. to overwrite config.yaml value

In [4]:
from nornir import InitNornir
nr = InitNornir(
    config_file="config.yaml",
    runner={
        "plugin": "threaded",
        "options": {
            "num_workers": 50,     # in this config.yaml "num_workers": 100
        },
    },
)
nr.config.runner.options["num_workers"]

50

[top](#top_of_page)

<a id="inventory"></a>
## 2 Inventory

+  MOST important piece of nornir

In [5]:
from nornir.core.inventory import Host
print(Host.schema())

{'name': 'str', 'connection_options': {'$connection_type': {'extras': {'$key': '$value'}, 'hostname': 'str', 'port': 'int', 'username': 'str', 'password': 'str', 'platform': 'str'}}, 'groups': ['$group_name'], 'data': {'$key': '$value'}, 'hostname': 'str', 'port': 'int', 'username': 'str', 'password': 'str', 'platform': 'str'}


+ JSON object, make easier to read with:

In [6]:
import json
print(json.dumps(Host.schema(), indent=4))

{
    "name": "str",
    "connection_options": {
        "$connection_type": {
            "extras": {
                "$key": "$value"
            },
            "hostname": "str",
            "port": "int",
            "username": "str",
            "password": "str",
            "platform": "str"
        }
    },
    "groups": [
        "$group_name"
    ],
    "data": {
        "$key": "$value"
    },
    "hostname": "str",
    "port": "int",
    "username": "str",
    "password": "str",
    "platform": "str"
}


+ hosts, groups **return** dictionaries, can be pretty printed as such

In [7]:
import pprint
print("Hosts:")
pprint.pprint(nr.inventory.hosts, indent=1)
print("-----\nGroups:")
pprint.pprint(nr.inventory.groups)

Hosts:
{'host1.bma': Host: host1.bma,
 'host1.cmh': Host: host1.cmh,
 'host2.bma': Host: host2.bma,
 'host2.cmh': Host: host2.cmh,
 'leaf00.bma': Host: leaf00.bma,
 'leaf00.cmh': Host: leaf00.cmh,
 'leaf01.bma': Host: leaf01.bma,
 'leaf01.cmh': Host: leaf01.cmh,
 'spine00.bma': Host: spine00.bma,
 'spine00.cmh': Host: spine00.cmh,
 'spine01.bma': Host: spine01.bma,
 'spine01.cmh': Host: spine01.cmh}
-----
Groups:
{'bma': Group: bma, 'cmh': Group: cmh, 'eu': Group: eu, 'global': Group: global}


+ or tap in, e.g.:

In [8]:
host = nr.inventory.hosts["leaf01.bma"]
for k,v in host.items():
    print(f"{k}: {v}")

site: bma
role: leaf
type: network_device
asn: 65100
domain: global.local


[top](#top_of_page)

<a id="inheritance"></a>
## 3 Inheritance

In [9]:
# initilize Nornir again with all inventory files
nr = InitNornir(config_file="config.yaml")

### 3.1 Example leaf01.bma host

Can be found in ./inventory/hosts.yaml as follows:<br>
<code>...
    leaf01.bma:
      hostname: 127.0.0.1
      username: vagrant
      password: wrong_password
      port: 12203
      platform: junos
      groups:
          - bma
      data:
          site: bma
          role: leaf
          type: network_device
</code>
and can be tapped in with normal dictonary methods:

Assign new **variable** with leaf01.bma

In [10]:
leaf01_bma = nr.inventory.hosts["leaf01.bma"]

In ./inverntory/groups.yaml bma is defined as:<br>
<code>
global:
    data:
        domain: global.local
        asn: 1
eu:
    data:
        asn: 65100
bma:
    groups:
        - eu
        - global
...
</code>

In [11]:
leaf01_bma["domain"]  # comes from the group `global` and has therefore this "domain" value

'global.local'

In [12]:
leaf01_bma["asn"]  # comes from group `eu` and has therefore this "asn" value

65100

---

### 3.2 Example leaf01.cmh host

+ If neither the host nor the parents have a specific value for a key, values in defaults will be returned.

<code>
leaf01.cmh:
    hostname: 127.0.0.1
    username: vagrant
    password: ""
    port: 12203
    platform: junos
    groups:
        - cmh
    data:
        site: cmh
        role: leaf
        type: network_device
        asn: 65101
</code>

In [13]:
leaf01_cmh = nr.inventory.hosts["leaf01.cmh"]
leaf01_cmh["domain"]  # comes from defaults

'acme.local'

### 3.3 Non-existent keys

+ python throws normal error, if the key is not existing

In [14]:
leaf01_cmh["wrong_key"]

KeyError: 'wrong_key'

In [15]:
# can be caught as usual:
try:
    leaf01_cmh["wrong_key"]
except KeyError as e:
    print(f"Couldn't find key: {e}")

Couldn't find key: 'wrong_key'


+ also when using this_nr.**data** the key won't be found

In [16]:
leaf01_cmh.data

{'site': 'cmh', 'role': 'leaf', 'type': 'network_device', 'asn': 65101}

In [17]:
try:
    leaf01_cmh.data["domain"]
except KeyError as e:
    print(f"Couldn't find key: {e}")

Couldn't find key: 'domain'


[top](#top_of_page)

<a id="filtering"></a>
## 4 Filtering

### 4.1 Basic filtering by Key-Value pairs

+ operate on groups of hosts

#### 4.1.1 One Pair

In [18]:
print(nr.filter(site="cmh").inventory.hosts.keys())

dict_keys(['host1.cmh', 'host2.cmh', 'spine00.cmh', 'spine01.cmh', 'leaf00.cmh', 'leaf01.cmh'])


#### 4.1.2 Multiple Pairs

In [19]:
print(nr.filter(site="cmh", role="spine").inventory.hosts.keys())

dict_keys(['spine00.cmh', 'spine01.cmh'])


#### 4.1.3 Culmulative Method

In [20]:
print(nr.filter(site="cmh").filter(role="spine").inventory.hosts.keys())

dict_keys(['spine00.cmh', 'spine01.cmh'])


#### 4.1.4 Per assigning variable and filter this

In [21]:
cmh = nr.filter(site="cmh")
print(cmh.inventory.hosts.keys())

dict_keys(['host1.cmh', 'host2.cmh', 'spine00.cmh', 'spine01.cmh', 'leaf00.cmh', 'leaf01.cmh'])


In [22]:
print(cmh.filter(role="spine").inventory.hosts.keys())
print(cmh.filter(role="leaf").inventory.hosts.keys())

dict_keys(['spine00.cmh', 'spine01.cmh'])
dict_keys(['leaf00.cmh', 'leaf01.cmh'])


#### 4.1.5 Return children of a group (retuned as set)

In [23]:
print(nr.inventory.children_of_group("eu"))
print(type(nr.inventory.children_of_group("eu")))   

{Host: leaf01.bma, Host: leaf00.bma, Host: spine00.bma, Host: spine01.bma, Host: host2.bma, Host: host1.bma}
<class 'set'>


### 4.2 Advanced Filtering

#### 4.2.1 Filter Functions - simple queries

The **filter_func** parameter can contain code to filter the hosts. The function signature is as simple as *my_func(host)* where host is an object of type Host and it has to return either **True** or **False** (to indicate if you want to host or not).

In [24]:
# create a function (in this case simply filter by length of name)

In [25]:
def has_long_name(host):
    """returns True or False if length of host == 11"""
    return len(host.name) == 11

In [26]:
nr.filter(filter_func=has_long_name).inventory.hosts.keys()

dict_keys(['spine00.cmh', 'spine01.cmh', 'spine00.bma', 'spine01.bma'])

In [27]:
# lambda function

In [28]:
nr.filter(filter_func=lambda x: len(x.name) == 11).inventory.hosts.keys()

dict_keys(['spine00.cmh', 'spine01.cmh', 'spine00.bma', 'spine01.bma'])

#### 4.2.2 Filter Object - complex queries

You can also use a filter objects to incrementally create a complex query objects.

In [29]:
# first you need to import the F object
from nornir.core.filter import F

In [30]:
# hosts in group cmh
cmh = nr.filter(F(groups__contains="cmh"))
print(cmh.inventory.hosts.keys())

dict_keys(['host1.cmh', 'host2.cmh', 'spine00.cmh', 'spine01.cmh', 'leaf00.cmh', 'leaf01.cmh'])


##### 4.2.2.1 Logic filtering

In [31]:
# devices running either linux or eos
linux_or_eos = nr.filter(F(platform="linux") | F(platform="eos"))
print(linux_or_eos.inventory.hosts.keys())

dict_keys(['host1.cmh', 'host2.cmh', 'spine00.cmh', 'leaf00.cmh', 'host1.bma', 'host2.bma', 'spine00.bma', 'leaf00.bma'])


In [32]:
# cmh devices that are not spines
cmh_and_not_spine = nr.filter(F(groups__contains="cmh") & ~F(role="spine"))
print(cmh_and_not_spine.inventory.hosts.keys())

dict_keys(['host1.cmh', 'host2.cmh', 'leaf00.cmh', 'leaf01.cmh'])


##### 4.2.2.2 Access nested Data

Access any nested data by separating the elements in the path with two underscores  __. Then use __contains to check if an element exists or if a string has a particular substring.



In [33]:
nested_string_asd = nr.filter(F(nested_data__a_string__contains="asd"))
print(nested_string_asd.inventory.hosts.keys())

dict_keys(['host1.cmh'])


In [34]:
a_dict_element_equals = nr.filter(F(nested_data__a_dict__c=3))
print(a_dict_element_equals.inventory.hosts.keys())

dict_keys(['host2.cmh'])


In [35]:
a_list_contains = nr.filter(F(nested_data__a_list__contains=2))
print(a_list_contains.inventory.hosts.keys())

dict_keys(['host1.cmh', 'host2.cmh'])


[top](#top_of_page)

<a id="tasks"></a>
## 5 Tasks

A kind of function that takes a Task as first paramater and returns a Result.<br>
(reusable piece of code that implements some functionality for a single host)

In [36]:
# run tasks on groups of hosts, i.e. initializing objects for later use
from nornir_utils.plugins.functions import print_result

nr = InitNornir(config_file="config.yaml")
# filtering objects to simplify output
nr = nr.filter(site="cmh", role="host")

#### 5.1 Simple function

In [37]:
from nornir.core.task import Task, Result
from nornir_utils.plugins.functions import print_result

def hello_world(task: Task) -> Result:
    """Return string with host name and hardcoded message"""
    return Result(
        host=task.host,
        result=f"{task.host.name} says hello world!"
    )

In [38]:
# To execute a task you can use the run method:
result = nr.run(task=hello_world)
print_result(result)

hello_world*********************************************************************
* host1.cmh ** changed : False *************************************************
vvvv hello_world ** changed : False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
host1.cmh says hello world!
^^^^ END hello_world ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
* host2.cmh ** changed : False *************************************************
vvvv hello_world ** changed : False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
host2.cmh says hello world!
^^^^ END hello_world ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^


#### 5.2 Simple function with additional parameters


+ use any number of arguments to extend their functionality

In [39]:
def say(task: Task, text: str = "default msg") -> Result:
    """Return string with host name and message, if no message is given 'default msg' is default"""
    return Result(
        host=task.host,
        result=f"{task.host.name} says {text}"
    )

+ but need to be called like before **with** specifying the values for the extra argument:
+ passing a **name** argument allows to give the task a descriptive name. (If it’s not specified the function name is used instead, e.g. hello_world in the 5.1 example above)

In [40]:
result = nr.run(
    name="Saying goodbye in a very friendly manner",   # "rename" the function, i.e. give it description 
    task=say,
    text="buhbye!"                                     # additional parameter
)
print_result(result)

Saying goodbye in a very friendly manner****************************************
* host1.cmh ** changed : False *************************************************
vvvv Saying goodbye in a very friendly manner ** changed : False vvvvvvvvvvvvvvv INFO
host1.cmh says buhbye!
^^^^ END Saying goodbye in a very friendly manner ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
* host2.cmh ** changed : False *************************************************
vvvv Saying goodbye in a very friendly manner ** changed : False vvvvvvvvvvvvvvv INFO
host2.cmh says buhbye!
^^^^ END Saying goodbye in a very friendly manner ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^


#### 5.3 Grouping tasks

Build more complex functionality by combining smaller building blocks, i.e. tasks calling other tasks.<br>
Example with the say function above an another count function defines as follows:

In [41]:
def count(task: Task, number: int) -> Result:
    """takes in a number and returns an f-string with list counting number elements"""
    return Result(
        host=task.host,
        result=f"{[n for n in range(0, number)]}"
    )

+ Build a new task combining these two functions

In [42]:
def greet_and_count(task: Task, number: int = 1,
                    greet: str = "default greet", bye: str = "default bye") -> Result:
    """uses the say and count functions, grouping the tasks with parameters for number, greet and bye
       (default: 1, "default greet" and "default bye" """
    task.run(  # call the say function
        name="The say function is called with the greet parameter",
        task=say,
        text=greet,
    )
    task.run(
        name="The count function is called ",
        task=count,
        number=number,
    )
    task.run(
        name="The say function is called with the bye parameter",
        task=say,
        text=bye,
    )    # the task (function) can have more code within itself, e.g. checking even or odd
    even_or_odds = "even" if (number+1) % 2 == 1 else "odd"
    return Result(
        host=task.host,
        result=f"{task.host} counted {even_or_odds} times!",
    )

In [43]:
this_num = 5
this_greet = "Hello, there"
this_bye = "Bye now"
result = nr.run(
    name=f"Counting to {this_num} and using the say function to greet and say bye",
    task=greet_and_count,
    number=this_num,
    greet=this_greet,
    bye=this_bye
)
print_result(result)

Counting to 5 and using the say function to greet and say bye*******************
* host1.cmh ** changed : False *************************************************
vvvv Counting to 5 and using the say function to greet and say bye ** changed : False  INFO
host1.cmh counted odd times!
---- The say function is called with the greet parameter ** changed : False ---- INFO
host1.cmh says Hello, there
---- The count function is called  ** changed : False -------------------------- INFO
[0, 1, 2, 3, 4]
---- The say function is called with the bye parameter ** changed : False ------ INFO
host1.cmh says Bye now
^^^^ END Counting to 5 and using the say function to greet and say bye ^^^^^^^^^
* host2.cmh ** changed : False *************************************************
vvvv Counting to 5 and using the say function to greet and say bye ** changed : False  INFO
host2.cmh counted odd times!
---- The say function is called with the greet parameter ** changed : False ---- INFO
host2.cmh says Hello, t

##### 5.3.1 Single results and tasks

+ Single Host (by tapping into result["host_name"])

In [44]:
print_result(result["host1.cmh"])

vvvv host1.cmh: Counting to 5 and using the say function to greet and say bye ** changed : False  INFO
host1.cmh counted odd times!
---- The say function is called with the greet parameter ** changed : False ---- INFO
host1.cmh says Hello, there
---- The count function is called  ** changed : False -------------------------- INFO
[0, 1, 2, 3, 4]
---- The say function is called with the bye parameter ** changed : False ------ INFO
host1.cmh says Bye now
^^^^ END Counting to 5 and using the say function to greet and say bye ^^^^^^^^^


+ Single task (by tapping into result["host_name"] and the *list* of tasks) 

In [45]:
print_result(result["host1.cmh"][2])

---- host1.cmh: The count function is called  ** changed : False --------------- INFO
[0, 1, 2, 3, 4]


[top](#top_of_page)

<a id="processing_results"></a>
## 6 Processing Results

Use the functions/tasks from above, but slightly adapt the <code>def say</code> to raise an error in a particular case.

In [46]:
import logging     # to log errors

In [47]:
def say_new(task: Task, text: str = "default msg", exception_host_name: str = "host2.cmh") -> Result:
    """Return string with host name and message, if no message is given 'default msg' is default, 
       to if the hostname equals exception_host_name, an error is raised."""
    if task.host.name == exception_host_name:
        raise Exception(f"An Exception was raised on host {exception_host_name}")
    return Result(
        host=task.host,
        result=f"{task.host.name} says {text}"
    )

In [48]:
def greet_and_count_new(task: Task, number: int = 1,
                    greet: str = "default greet", bye: str = "default bye") -> Result:
    """uses the say and count functions, grouping the tasks with parameters for number, greet and bye
       (default: 1, "default greet" and "default bye" """
    task.run(  # call the say function
        name="The say function is called with the greet parameter",
        severity_level=logging.DEBUG,
        task=say_new,
        text=greet,
    )
    task.run(
        name="The count function is called ",
        task=count,
        number=number,
    )
    task.run(
        name="The say function is called with the bye parameter",
        severity_level=logging.DEBUG,
        task=say_new,
        text=bye,
    )    # the task (function) can have more code within itself, e.g. checking even or odd
    even_or_odds = "even" if (number+1) % 2 == 1 else "odd"
    return Result(
        host=task.host,
        result=f"{task.host} counted {even_or_odds} times!",
    )

In [49]:
# re-instantiate the nr object and filter to cmh 
nr = InitNornir(config_file="config.yaml")
cmh = nr.filter(site="cmh", type="host")

#### 6.1 Simple approach

In [50]:
# run the adapted scripts
this_num = 5
this_greet = "Hello, there"
this_bye = "Bye now"
result = cmh.run(
    name=f"Counting to {this_num} and using the say function to greet and say bye",
    task=greet_and_count_new,
    number=this_num,
    greet=this_greet,
    bye=this_bye
)
print_result(result)

Counting to 5 and using the say function to greet and say bye*******************
* host1.cmh ** changed : False *************************************************
vvvv Counting to 5 and using the say function to greet and say bye ** changed : False  INFO
host1.cmh counted odd times!
---- The count function is called  ** changed : False -------------------------- INFO
[0, 1, 2, 3, 4]
^^^^ END Counting to 5 and using the say function to greet and say bye ^^^^^^^^^
* host2.cmh ** changed : False *************************************************
vvvv Counting to 5 and using the say function to greet and say bye ** changed : False  ERROR
Subtask: The say function is called with the greet parameter (failed)

---- The say function is called with the greet parameter ** changed : False ---- ERROR
Traceback (most recent call last):
  File "C:\Users\roman\Python\anaconda\envs\nornir_env\lib\site-packages\nornir\core\task.py", line 99, in start
    r = self.task(self, **self.params)
  File "C:\User

+ When *print_result* now, not all tasks are printed due to the logged <code>severity_level=logging.DEBUG</code>.<br>
By **default** only the **info level** info will be printed<br>
A failed task will always have its severity level changed to ERROR regardless of the one specified by the user. With the exception_host_name parameter an error was raised at the host, so it will **NOT** be printed, if not specified.

In [51]:
print_result(result["host1.cmh"])

vvvv host1.cmh: Counting to 5 and using the say function to greet and say bye ** changed : False  INFO
host1.cmh counted odd times!
---- The count function is called  ** changed : False -------------------------- INFO
[0, 1, 2, 3, 4]
^^^^ END Counting to 5 and using the say function to greet and say bye ^^^^^^^^^


+ change the severity_level by defining parameter in *print_result*

In [52]:
print_result(result, severity_level=logging.DEBUG)

Counting to 5 and using the say function to greet and say bye*******************
* host1.cmh ** changed : False *************************************************
vvvv Counting to 5 and using the say function to greet and say bye ** changed : False  INFO
host1.cmh counted odd times!
---- The say function is called with the greet parameter ** changed : False ---- DEBUG
host1.cmh says Hello, there
---- The count function is called  ** changed : False -------------------------- INFO
[0, 1, 2, 3, 4]
---- The say function is called with the bye parameter ** changed : False ------ DEBUG
host1.cmh says Bye now
^^^^ END Counting to 5 and using the say function to greet and say bye ^^^^^^^^^
* host2.cmh ** changed : False *************************************************
vvvv Counting to 5 and using the say function to greet and say bye ** changed : False  ERROR
Subtask: The say function is called with the greet parameter (failed)

---- The say function is called with the greet parameter ** chan

#### 6.2 Programmatic approach

The task groups will return an **AggregatedResult**.

In [53]:
result

AggregatedResult (Counting to 5 and using the say function to greet and say bye): {'host1.cmh': MultiResult: [Result: "Counting to 5 and using the say function to greet and say bye", Result: "The say function is called with the greet parameter", Result: "The count function is called ", Result: "The say function is called with the bye parameter"], 'host2.cmh': MultiResult: [Result: "Counting to 5 and using the say function to greet and say bye", Result: "The say function is called with the greet parameter"]}

This is a dict-like object which can be used to iterate over or access hosts directly.

In [54]:
result.keys()

dict_keys(['host1.cmh', 'host2.cmh'])

In [55]:
result["host1.cmh"]

MultiResult: [Result: "Counting to 5 and using the say function to greet and say bye", Result: "The say function is called with the greet parameter", Result: "The count function is called ", Result: "The say function is called with the bye parameter"]

Each **AggregatedResult** contains a *list*-like **MultiResult** object, therefore can be indexed.

In [56]:
result["host1.cmh"][0]

Result: "Counting to 5 and using the say function to greet and say bye"

Each result also contains the **changed** and **failed** from the respective host. Therefore, it is possible to return the directly.

In [57]:
for host in ["host1.cmh", "host2.cmh"]:
    print(f"{host} -> changed: {result[host].changed}, failed: {result[host].failed}")


host1.cmh -> changed: False, failed: False
host2.cmh -> changed: False, failed: True


[top](#top_of_page)

## 7 Failed Tasks

#### 7.1 Basics

+ the **.failed** property will be set **True** when a task failed

In [58]:
print(result.failed)

True


+ under **.failed_hosts** is a dict-loke object with the failed hosts

In [59]:
print(result.failed_hosts)

{'host2.cmh': MultiResult: [Result: "Counting to 5 and using the say function to greet and say bye", Result: "The say function is called with the greet parameter"]}


+ the list-like onject **.exception** contains the Exception message under index 1

In [60]:
print(result['host2.cmh'].exception)
print(result['host2.cmh'][1].exception)

Subtask: The say function is called with the greet parameter (failed)

An Exception was raised on host host2.cmh


+ **NornirExecutionError**<br>Built-in method raising an exception if the task had an error

In [61]:
from nornir.core.exceptions import NornirExecutionError
try:
    result.raise_on_error()
except NornirExecutionError:
    print("An error was raised.")

An error was raised.


#### 7.2 Skipped hosts<br>
When re-instating cmh, there was an error raised on host2.cmh.<br>
A set of failed hosts is keeping track of failed host in shared data object **nr.data.failed_hosts**:

In [62]:
print(nr.data.failed_hosts)

{'host2.cmh'}


+ That way, **failed hosts** will be tracked and future tasks **WILL NOT** be run on them by Nornir.

In [63]:
# New task
def new_task(task: Task) -> Result:
    """similar to say, to show failed tasks handling"""
    return Result(host=task.host, result=f"{task.host.name}: new task was run on.")

In [64]:
result = cmh.run(task=new_task)

In [65]:
print_result(result)

new_task************************************************************************
* host1.cmh ** changed : False *************************************************
vvvv new_task ** changed : False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
host1.cmh: new task was run on.
^^^^ END new_task ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^


+ To **include failed** hosts, you can set parameter **on_failed** to True on calling the .run function (default is False):

In [66]:
result = cmh.run(task=new_task, on_failed=True)
print_result(result)

new_task************************************************************************
* host1.cmh ** changed : False *************************************************
vvvv new_task ** changed : False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
host1.cmh: new task was run on.
^^^^ END new_task ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
* host2.cmh ** changed : False *************************************************
vvvv new_task ** changed : False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
host2.cmh: new task was run on.
^^^^ END new_task ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^


+ To **exclude "good"** hosts, you can set parameter **on_good** to False on calling the .run function (default is True):

In [67]:
result = cmh.run(task=new_task, on_failed=True, on_good=False)
print_result(result)

new_task************************************************************************
* host2.cmh ** changed : False *************************************************
vvvv new_task ** changed : False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
host2.cmh: new task was run on.
^^^^ END new_task ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^


#### 7.3 Resetting .failed_hosts

Make flagged hosts eligible for future tasks again by resetting the list completely with reset_failed_hosts:

In [68]:
nr.data.reset_failed_hosts()
print(nr.data.failed_hosts)

set()


(lookup recover_host for individually resetting hosts)

[top](#top_of_page)

<a id="processors"></a>
## 8 Processors

Processors are plugins that can execute code on certain events.<br>
Alternative way of dealing with the results of a task, with the following advantages:<br>
+ Due to its event-based nature, the events can be processed asynchronously, meaning the result will be processed of a host exactly once the host is completed without the need to wait for the rest of the hosts to complete.
+ Tapping into events code allows for more concise and easier to understanding.


In [69]:
from typing import Dict # to annotate code with types

In [70]:
# re-instantiate the nr object
nr = InitNornir(config_file="config.yaml")

In [71]:
from nornir.core import Nornir
from nornir.core.inventory import Host
from nornir.core.task import AggregatedResult, MultiResult, Result, Task

class PrintResult:
    def task_started(self, task: Task) -> None:
        print(f">>> starting: {task.name}")

    def task_completed(self, task: Task, result: AggregatedResult) -> None:
        print(f">>> completed: {task.name}")

    def task_instance_started(self, task: Task, host: Host) -> None:
        pass

    def task_instance_completed(
        self, task: Task, host: Host, result: MultiResult
    ) -> None:
        print(f"  - {host.name}: - {result.result}")

    def subtask_instance_started(self, task: Task, host: Host) -> None:
        pass  # to keep example short and sweet we ignore subtasks

    def subtask_instance_completed(
        self, task: Task, host: Host, result: MultiResult
    ) -> None:
        pass  # to keep example short and sweet we ignore subtasks

In [72]:
class SaveResultToDict:
    def __init__(self, data: Dict[str, None]) -> None:
        self.data = data

    def task_started(self, task: Task) -> None:
        self.data[task.name] = {}
        self.data[task.name]["started"] = True

    def task_completed(self, task: Task, result: AggregatedResult) -> None:
        self.data[task.name]["completed"] = True

    def task_instance_started(self, task: Task, host: Host) -> None:
        self.data[task.name][host.name] = {"started": True}

    def task_instance_completed(
        self, task: Task, host: Host, result: MultiResult
    ) -> None:
        self.data[task.name][host.name] = {
            "completed": True,
            "result": result.result,
        }

    def subtask_instance_started(self, task: Task, host: Host) -> None:
        pass  # to keep example short and sweet we ignore subtasks

    def subtask_instance_completed(
        self, task: Task, host: Host, result: MultiResult
    ) -> None:
        pass  # to keep example short and sweet we ignore subtasks

In [73]:
def greeter(task: Task, greet: str) -> Result:
    """simple function like say, s.a."""
    return Result(host=task.host, result=f"{greet}! my name is {task.host.name}")

+ similary to .filter, with_processors returns a copy of the nornir object but with the processors assigned to it. use the method to assign both processors on both classes created above

In [74]:
# NBVAL_IGNORE_OUTPUT

data = {}  # SaveResultToDict class object will store the information


nr_with_processors = nr.with_processors([SaveResultToDict(data), PrintResult()])

# now we can use nr_with_processors to execute our greeter task
nr_with_processors.run(
    name="hi!",
    task=greeter,
    greet="hi",
)
nr_with_processors.run(
    name="bye!",
    task=greeter,
    greet="bye",
)

>>> starting: hi!
  - host1.cmh: - hi! my name is host1.cmh  - host2.cmh: - hi! my name is host2.cmh
  - spine00.cmh: - hi! my name is spine00.cmh  - spine01.cmh: - hi! my name is spine01.cmh
  - leaf00.cmh: - hi! my name is leaf00.cmh  - leaf01.cmh: - hi! my name is leaf01.cmh
  - host1.bma: - hi! my name is host1.bma
  - host2.bma: - hi! my name is host2.bma


  - spine00.bma: - hi! my name is spine00.bma
  - spine01.bma: - hi! my name is spine01.bma  - leaf00.bma: - hi! my name is leaf00.bma  - leaf01.bma: - hi! my name is leaf01.bma



>>> completed: hi!
>>> starting: bye!
  - host1.cmh: - bye! my name is host1.cmh  - host2.cmh: - bye! my name is host2.cmh
  - spine00.cmh: - bye! my name is spine00.cmh  - spine01.cmh: - bye! my name is spine01.cmh
  - leaf00.cmh: - bye! my name is leaf00.cmh
  - leaf01.cmh: - bye! my name is leaf01.cmh  - host1.bma: - bye! my name is host1.bma  - host2.bma: - bye! my name is host2.bma  - spine00.bma: - bye! my name is spine00.bma



  - spine01.bma

AggregatedResult (bye!): {'host1.cmh': MultiResult: [Result: "bye!"], 'host2.cmh': MultiResult: [Result: "bye!"], 'spine00.cmh': MultiResult: [Result: "bye!"], 'spine01.cmh': MultiResult: [Result: "bye!"], 'leaf00.cmh': MultiResult: [Result: "bye!"], 'leaf01.cmh': MultiResult: [Result: "bye!"], 'host1.bma': MultiResult: [Result: "bye!"], 'host2.bma': MultiResult: [Result: "bye!"], 'spine00.bma': MultiResult: [Result: "bye!"], 'spine01.bma': MultiResult: [Result: "bye!"], 'leaf00.bma': MultiResult: [Result: "bye!"], 'leaf01.bma': MultiResult: [Result: "bye!"]}

All messages on screen were printed by the processor **PrintResult**.<br>
The returned AggregatedResult is not used (needed) it here.

The **SaveToResults** class saved the data to the **data** dictionary:

In [75]:
import json
print(json.dumps(data, indent=4))

{
    "hi!": {
        "started": true,
        "host1.cmh": {
            "completed": true,
            "result": "hi! my name is host1.cmh"
        },
        "host2.cmh": {
            "completed": true,
            "result": "hi! my name is host2.cmh"
        },
        "spine00.cmh": {
            "completed": true,
            "result": "hi! my name is spine00.cmh"
        },
        "spine01.cmh": {
            "completed": true,
            "result": "hi! my name is spine01.cmh"
        },
        "leaf00.cmh": {
            "completed": true,
            "result": "hi! my name is leaf00.cmh"
        },
        "leaf01.cmh": {
            "completed": true,
            "result": "hi! my name is leaf01.cmh"
        },
        "host1.bma": {
            "completed": true,
            "result": "hi! my name is host1.bma"
        },
        "host2.bma": {
            "completed": true,
            "result": "hi! my name is host2.bma"
        },
        "spine00.bma": {
           

In [76]:
data["hi!"]["host1.cmh"]

{'completed': True, 'result': 'hi! my name is host1.cmh'}

[top](#top_of_page)